# multi agent 지식알림 챗봇 만들기를 위한 연습입니다^^
- LangChain 공식 docs 중 Use cases/Tool use and agents의 문서 사용
- 중간중간 공식 docs와 다른 시도가 끼어 있을 수 있습니다......?

In [6]:
import getpass
import os

In [7]:
from dotenv import load_dotenv

load_dotenv('../dot.env')

True

# Tool 만들기
- 정해져 있는 tool 말고도 필요에 따른 사용자 정의 tool 구성 가능
- @tool 아래 라인에 함수 형태로 정의
- 아래 셀 참조

In [8]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers together.""" #-> tool에 대한 description
    return a * b



# Tool의 메타데이터 출력

In [9]:
print(multiply.name)  # tool의 이름을 출력합니다.
print(multiply.description)  # tool의 설명을 출력합니다.
print(multiply.args)  # tool의 인자 정보를 출력합니다.

multiply
Multiply two numbers together.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


# Tool에 인자를 넣을 때 잘 들어가지 않는다면???
- print('tool_name'.args) 후 인자 살펴보기 
- 반드시 위 함수 실행 결과에 따른 형식에 따라 입력해야함

In [10]:
multiply.invoke({"a" : 2, "b" : 3})

6

# 간단한 Chain 만들어.보기 
![Agent_Tool_Chains](imgs/Agent_tool_Chain.png)
- 예제에서는 chatgpt를 사용, but 본인은 xionic에서 공개한 llama-3-70B를 쓰기로 했습니다.
- 여기저기서 쓰다보면 토큰이 무지하게 나오니까....



In [15]:
# LLM 정의
from langchain_openai import ChatOpenAI
llama3 = ChatOpenAI(
    base_url = 'http://sionic.chat:8001/v1',
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
    model = 'xionic-ko-llama-3-70b',
    temperature = 0.1,
)
# dolphine = ChatOpenAI(
#     base_url = 'http://localhost:1234/v1',
#     api_key = "lm-studio",
#     model = 'cognitivecomputations/dolphin-2.9-llama3-8b-gguf',
#     temperature = 0,
# )

llm = ChatOpenAI(model = 'gpt-3.5-turbo')



# LLM과 정의된 TOOL을 묶는법
# LLM.bind_tools(['tool_name'])

In [17]:
# LLM과 정의된 TOOL을 bind
llama3_with_tools = llama3.bind_tools([multiply])
llm_with_tools = llm.bind_tools([multiply])
# dolphine_with_tools = dolphine.bind_tools([multiply])

# message를 넣어 LLM이 tool에 제대로 query를 넣는지 확인
msg = llm_with_tools.invoke("2와 3을 곱해줘")
# msg_dolphine = dolphine_with_tools.invoke("2와 3을 곱해줘")
msg_llama_3 = llama3_with_tools.invoke("2와 3을 곱해줘")
print(msg_llama_3, '/n')
print(msg)
# print('\n', msg_dolphine)

content='2 × 3 = 6' response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 20, 'total_tokens': 28}, 'model_name': 'xionic-ko-llama-3-70b', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-26ac7d06-9ed9-459c-a9a3-4f60e7580ad0-0' usage_metadata={'input_tokens': 20, 'output_tokens': 8, 'total_tokens': 28} /n
content='' additional_kwargs={'tool_calls': [{'id': 'call_1aUx0VHQtrjqrEETRK4nKP7H', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 56, 'total_tokens': 73}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-51f23efe-639f-45a9-bd9d-b38527401063-0' tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_1aUx0VHQtrjqrEETRK4nKP7H'}] usage_metadata={'input_tokens': 56, 'output_tokens': 17, 'total_tokens': 73}


In [43]:
llm_with_tools.__dict__

{'name': None,
 'bound': ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x3254c3c50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x3254ccdd0>, openai_api_key=SecretStr('**********'), openai_proxy=''),
 'kwargs': {'tools': [{'type': 'function',
    'function': {'name': 'multiply',
     'description': 'multiply(a: int, b: int) -> int - Multiply two numbers together.',
     'parameters': {'type': 'object',
      'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}},
      'required': ['a', 'b']}}}]},
 'config': {},
 'config_factories': [],
 'custom_input_type': None,
 'custom_output_type': None}

In [44]:
msg.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_bdk6KpSy9uCpDtzyYcUsXU0V'}]

## Llama-3-xionic의 경우, bind_tools method 적용 시, tool을 제대로 인식하지 못하는 것 확인
## bind_tools method 사용 부분은 chatgpt-3.5-turbo 사용

In [45]:
msg.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_bdk6KpSy9uCpDtzyYcUsXU0V'}]

# Invoking the tool
- 훌륭해요! 우리는 도구 호출을 생성할 수 있습니다. 그러나 실제로 도구를 호출하고 싶다면 어떻게 해야 할까요? 이를 위해 생성된 도구 인자를 우리의 도구에 전달해야 합니다. 간단한 예로, 첫 번째 tool_call의 인자를 추출해 보겠습니다:


In [46]:
from operator import itemgetter

# llm_with_tools에서 tool_calls의 첫 번째 호출을 가져와서 그 인자들을 multiply 함수에 전달합니다.
# -> 그렇다네요 chain의 출력값 연쇄는 항상 헷갈립니다 허허....
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
chain.invoke('23*4는뭐야?')

92

# Agents
![Agent_Tool_Agents](imgs/Agent_tool_Agents.png)
- 특정 사용자 입력에 필요한 도구 사용 순서를 알고 있을 때 체인은 매우 유용합니다. 그러나 특정 사용 사례에서는 입력에 따라 도구를 사용하는 횟수가 달라집니다. 이러한 경우에는 모델이 도구를 몇 번 사용할지, 어떤 순서로 사용할지 스스로 결정하게 하고 싶습니다. 에이전트는 바로 이를 가능하게 합니다.
- LangChain은 다양한 사용 사례에 최적화된 여러 내장 에이전트를 제공합니다. 여기에서 모든 에이전트 유형에 대해 읽어보세요. 
- 우리는 도구 호출 에이전트를 사용할 것입니다. 이는 일반적으로 가장 신뢰할 수 있는 종류로, 대부분의 사용 사례에 권장됩니다.


In [47]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.agents import create_json_chat_agent
# xionic의 llama-3 모델으로 agent 생성 시 create_json_agent 메소드를 사용해야하며,
# AgentExecutor를 사용해야함
# json prompt-ko -> 테디노트님 작품
json_prompt_ko = hub.pull('teddynote/react-chat-json-korean')
# prompt 의 messages를 출력합니다.
json_prompt_ko.messages
# json_prompt = hub.pull("hwchase17/react-chat-json") 
# json_prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

# Agents는 여러 도구를 사용하기 쉽다는 점에서 장점이 있습니다.

In [48]:
@tool
def add(a: int, b: int) -> int:
    """두 수를 더합니다."""
    tool_name = 'exponentiate'
    return a + b

@tool
def exponentiate(a: int, b: int) -> int:
    """두 수를 거듭제곱합니다."""
    tool_name = 'exponentiate'
    return a ** b

tools = [add, 
         exponentiate, 
         multiply 
         ]

# tool_calling_agent는 여러 도구를 사용하여 사용자의 요청을 처리할 수 있는 에이전트입니다.
- 이 에이전트는 사용자의 입력을 분석하고, 필요한 도구를 호출하여 적절한 작업을 수행합니다.
- 예를 들어, 계산이나 데이터 검색과 같은 작업을 자동으로 처리할 수 있습니다.

In [49]:
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
# llama_3_agent = create_json_chat_agent(llama3, tools, json_prompt_ko) #xionic-ko-llama-3-70b 사용

In [50]:
# llama3_agent_executor = AgentExecutor(agent = llama_3_agent, 
#                                tools = tools, 
#                                verbose=True,
#                                handle_parsing_errors = True,
#                                return_intermediate_steps = True
#                                )
agent_executor = AgentExecutor(agent = agent, 
                               tools = tools, 
                               verbose=True,
                               )

In [97]:
response = agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)
print(f'답변: {response["output"]}')



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'a': 3, 'b': 5}`


243
Invoking: `add` with `{'a': 12, 'b': 3}`


15
Invoking: `multiply` with `{'a': 243, 'b': 15}`


3645
Invoking: `exponentiate` with `{'a': 3645, 'b': 2}`


13286025The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025.

> Finished chain.
답변: The result of taking 3 to the fifth power and multiplying that by the sum of twelve and three, then squaring the whole result is 13,286,025.
